In [ ]:
import pandas as pd
import numpy as np 
import datetime
import re
from math import floor
from datetime import datetime,date,timedelta

In [ ]:
data_path = '..//Data//'
#### Input Outage: ##################
# fn = 'Ticketed_Outage_Spring_21Jun.csv'
# fn = 'Ticketed_Outage_Fall_27Feb.csv'
# fn = '1398_Ticketed_Outage_Winter.csv'
# fn = 'Ticketed_Outage_Spring_1399.csv'
fn = 'Ticketed_Outage_Summer_till31July.csv'

df_sn = pd.read_excel('../Data/Update_SiteNames.xlsx')
df_all = pd.DataFrame()
        
df = pd.read_csv(data_path +  fn)
df = df[['CellName', 'Location', 'Province', 'NE', 'StartOfOutage', 'EndOfOutage', 'Duration', 'Ddif', 'OG', 'CTAG', 'outage_number',\
       'outage_tag_number', 'Cell_Outage_Number', 'OutageGroup', 'Impact', 'LevelTag', 'ENTRY_TicketNumber', 'CC_TicketNumber', 'HC_TicketNumber',\
       'PE_TicketNumber', 'Alias_TicketNumber', 'CountOfOutageInGroup', 'Tag_Number', 'TicketTag', 'TicketGroup', 'TicketNumber',\
       'ExactMatchTicket', 'TicketTitle', 'TicketClosureType', 'RootCause', 'MainTicket', 'TicketCreate', 'TicketCreate_CoCurrent',\
       'Impact_Start_Date', 'Impact_Start_Date_ConCurrent', 'EventEnd', 'Health_Check', 'Concurrent_Tag', 'Exclude_Cause', 'Exclusion_Type',\
       'ST', 'IT', 'MTCT','Comments']] # Last JT Comment

print(len(df))
#####################################
df['CellName'] = df['CellName'].astype(str)
df['CellName'] = df['CellName'].str.upper()
df['NE']=df['NE'].str.upper()
df.loc[df['CellName'].str[:2] == 'XH' , 'CellName'] = 'TH' + df.loc[df['CellName'].str[:2] == 'XH' , 'CellName'].str[2:]

## Regular expression(Regex)
pat_name = '\A[A-Z]{2}[1-9]*[GLU]{0,3}[0-9]{4}'#'[A-F]{1}'
pat_tech = '[1-9]{1}[GLU]{1}'
df['Sector'] = df['CellName'].apply(lambda x:  ''.join(re.findall(pat_name, x)))
df = df[df['Sector'] != '']
#Create SiteName
df['Site']=np.where(len(df['Sector']) > 7, df['Sector'].str[:8], df['Sector'].str[:6])

df['Location'] = df['Site'].apply(lambda x: re.sub(pat_tech,'',x))
df = df[df['Location'] != '']

df['Province'] = df['Site'].str[:2]
############################################################################
df['Tech'] = '2G'
df['CellName'] = df['CellName'].str.upper()

df.loc[df['CellName'].str[3]=='L','Tech'] = '4G'
df.loc[df['CellName'].str[3]=='U','Tech'] = '3G'
df.loc[df['CellName'].str[3]=='G','Tech'] = '2G'
df.loc[df['CellName'].str[3:6]=='GUL','Tech'] = '4G'

df.loc[df['Tech'] == '4G','NE'] = df.loc[df['Tech'] == '4G','NE'] + '_LTE' 

df = df[['CellName','Site','Location','Province','Tech','NE','StartOfOutage','Comments',\
        'EndOfOutage','Duration','ENTRY_TicketNumber','TicketTitle','TicketClosureType','RootCause','Exclude_Cause', 'Impact', 'LevelTag']]

df['Down_Time(Days)'] = df['Duration']/(24*3600)
df['Down_Time(Days)'] = df['Down_Time(Days)'].apply(np.floor)

df = df.merge(df_sn[['Location','SiteName']],  left_on = 'Location', right_on = 'Location', how = 'left')

aras = pd.read_excel('..//Data/Update_Aras.xlsx')
df = df.merge(aras[['Location','Latitude','Longitude']], on = 'Location', how= 'left')

df = df.rename(columns={'Comments' : 'Last Job Ticket Comment'}) #Last JT Comment

# df.to_csv('New_Data/spring_old.csv', index=False, encoding='utf-8-sig')
# df.to_csv('New_Data/fall_new.csv', index=False, encoding='utf-8-sig')
# df.to_csv('New_Data/winter_new.csv', index=False, encoding='utf-8-sig')
# df.to_csv('New_Data/spring_new.csv', index=False, encoding='utf-8-sig')
df.to_csv('New_Data/summer_new.csv', index=False, encoding='utf-8-sig')

In [ ]:
data_path = 'New_Data/'
D_spring99 = 'spring_new.csv'
D_summer = 'summer_new.csv'

spring99 = pd.read_csv(data_path + D_spring99)
summer99 = pd.read_csv(data_path + D_summer)

spring99 = spring99.rename(columns={'Last JT Comment' : 'Last Job Ticket Comment'}) #Last JT Comment
summer99 = summer99.rename(columns={'Comments' : 'Last Job Ticket Comment'}) #Last JT Comment

summer99['broken'] = 0
spring99['broken'] = 0

summer99.loc[summer99['EndOfOutage'] == '2020-06-20 23:00:00.000','broken'] = 1
spring99.loc[spring99['StartOfOutage'] == '2020-06-21 00:00:00.000','broken'] = 1

su_sp = pd.concat([summer99,spring99], axis = 0, sort = True)

df_T = su_sp[su_sp['broken'] == 1].copy()
df_T.sort_values(by = ['CellName','StartOfOutage','EndOfOutage'], inplace = True)

df_T2 = df_T.groupby('CellName', as_index = False).agg({'StartOfOutage':'min','EndOfOutage':'max','Duration' : 'sum'})

df_T.drop(['StartOfOutage','EndOfOutage','Duration'], axis = 1, inplace = True)
df_T = df_T.merge(df_T2, left_on = ['CellName'], right_on = ['CellName'])
df_T.drop_duplicates(subset =["CellName","StartOfOutage","EndOfOutage","Duration"],keep = 'first', inplace = True)

su_sp = su_sp[su_sp['broken']==0]
su_sp = pd.concat([su_sp,df_T], axis = 0, sort = True)

su_sp['Down_Time(Days)'] = su_sp['Duration']/(24*3600)
su_sp['Down_Time(Days)'] = su_sp['Down_Time(Days)'].apply(np.floor)


wao_flag = True
if wao_flag:
    Total_seasons = su_sp.copy()
    #Total_seasons = Total_seasons[Total_seasons['Duration']>=(3*60)] ## Duration > 3 min
    Total_seasons['Down_Time(hour)'] = Total_seasons['Duration']/(1*3600)
    Total_seasons['Down_Time(hour)'] = Total_seasons['Down_Time(hour)'].apply(np.floor)
    Total_seasons.to_csv('Outputs/Ticket_DetailOutage(Seconds).csv', index=False, encoding='utf-8-sig')
else:
    Total_seasons = su_sp[su_sp['Duration']>=(24*3600)]
    Total_seasons.to_csv('Outputs/Ticket_DetailOutage(Daily).csv', index=False, encoding='utf-8-sig')

### ROOT CAUSE:

In [ ]:
######## Root Cause: Ticket Open ###################
ticket_open = pd.read_excel('Inputs/Ticket Open_31July.xls',sheet_name='Sheet1') #Read Ticket Open
Ticket_DetailOutage = pd.read_csv(r'Outputs/Ticket_DetailOutage(Seconds).csv') #Read Ticket_DetailOutage(Seconds)
# Ticket_DetailOutage = pd.read_csv(r'Outputs/Ticket_DetailOutage(Daily).csv') #Read Ticket_DetailOutage(Daily)

# Step 1: List Ticket Open
ticket_open['Tick'] = 1
Ticket_DetailOutage = Ticket_DetailOutage.merge(ticket_open[['Tick','TICKET_NUMBER']],left_on=['ENTRY_TicketNumber'],right_on=['TICKET_NUMBER'],how='left')
Ticket_DetailOutage.drop('TICKET_NUMBER',axis = 1,inplace=True)
Ticket_DetailOutage['Tick'] = Ticket_DetailOutage['Tick'].fillna(0)
Ticket_DetailOutage.loc[((Ticket_DetailOutage['TicketClosureType'].isnull() == True) & (dfo['Tick'] == 1)) , 'TicketClosureType'] = 'OTHERS-TICKET OPEN' 

# Step 2: Outages last day without review
dfo1 = Ticket_DetailOutage[['TicketClosureType','EndOfOutage']]
flag = ((dfo1['TicketClosureType'].isnull() == True) | (dfo1['TicketClosureType'] == 'OTHERS-UNKNOWN ISSUE') |(dfo1['TicketClosureType'] == 'OTHERS-ESCALATE TO VENDOR') |\
       (dfo1['TicketClosureType'] == 'OTHERS-HC-FO-UNKNOWN ISSUE') |(dfo1['TicketClosureType'] == 'OTHERS-AUTO RESTORE'))
flag2 = (dfo1['EndOfOutage'] == Ticket_DetailOutage['EndOfOutage'].max())
dfo1['Ticket'] = 0
dfo1.loc[(flag & flag2) , 'Ticket'] = 1 
dfo1.loc[dfo1['Ticket'] == 1 , 'TicketClosureType'] = 'OTHERS-TICKET OPEN' 
Ticket_DetailOutage['TicketClosureType'] = dfo1['TicketClosureType']
Ticket_DetailOutage.loc[Ticket_DetailOutage['TicketClosureType'].isnull()==True, 'TicketClosureType'] = 'UNKNOWN'
Ticket_DetailOutage.loc[Ticket_DetailOutage['TicketClosureType'] == 'UNKNOWN', 'RootCause'] = 'UNKNOWN ISSUE'
Ticket_DetailOutage.loc[Ticket_DetailOutage['TicketClosureType'] == 'OTHERS-TICKET OPEN', 'RootCause'] = 'TICKET OPEN'
######## Root Cause: BACK OFFICE ACTION ###################
Ticket_DetailOutage.loc[((Ticket_DetailOutage['RootCause'] == 'BACK OFFICE ACTION') & ((Ticket_DetailOutage['TicketClosureType'] == 'UNKNOWN') | (Ticket_DetailOutage['TicketClosureType'].isnull() == True))), 'TicketClosureType'] = 'BACK OFFICE ACTION'

######## Root Cause:   ###################
Ticket_DetailOutage.loc[Ticket_DetailOutage['TicketTitle'].str[0:2] == 'NI', 'TicketClosureType'] = 'NEW SITE' # & (dfo['TicketClosureType']!='OTHERS-TICKET OPEN')
Ticket_DetailOutage.loc[((Ticket_DetailOutage['TicketClosureType'] == "NEW SITE")) , 'RootCause'] = 'NEW SITE'
Ticket_DetailOutage.loc[((Ticket_DetailOutage['TicketClosureType'] == "OTHERS-VANDALISM")) , 'RootCause'] = 'VANDALISM' 
Ticket_DetailOutage.loc[((Ticket_DetailOutage['TicketClosureType'] == "ENVIRONMENT- TCT AIR CONDITION ISSUE") | \
         (Ticket_DetailOutage['TicketClosureType'] == "ENVIRONMENT-FREE COOLING FAULTY") | \
         (Ticket_DetailOutage['TicketClosureType'] == "ENVIRONMENT-OUTDOOR BTS COOLING SYSTEM")) , 'RootCause'] = 'ENVIRONMENT'

### Shamsi Date:

In [ ]:
from datetime import datetime,date,timedelta
import dateutil.parser


Dim = pd.read_csv('Full_DimDate.csv')   
Dim['Miladi'] = pd.to_datetime(Dim['Miladi']).dt.date

Ticket_DetailOutage['StartDate'] = pd.to_datetime(Ticket_DetailOutage['StartOfOutage']).dt.date
Ticket_DetailOutage['EndDate'] = pd.to_datetime(Ticket_DetailOutage['EndOfOutage']).dt.date

Ticket_DetailOutage = Ticket_DetailOutage.merge(Dim[['Miladi','Jalali']],left_on=['StartDate'],right_on=['Miladi'],how='left')
Ticket_DetailOutage.drop(['StartDate','Miladi'],axis = 1,inplace=True)
Ticket_DetailOutage.rename(columns={'Jalali': 'StartOfOutageP'}, inplace = True)

Ticket_DetailOutage = Ticket_DetailOutage.merge(Dim[['Miladi','Jalali']],left_on=['EndDate'],right_on=['Miladi'],how='left')
Ticket_DetailOutage.drop(['EndDate','Miladi'],axis = 1,inplace=True)
Ticket_DetailOutage.rename(columns={'Jalali': 'EndOfOutageP'}, inplace = True)

### POWER TYPE

In [ ]:
BTS_Nodes = pd.read_excel('BTS-Nodes-202005301211.xlsx',sheet_name='Location')
BTS_Nodes = BTS_Nodes[['نوع برق','کد سایت','نوع لینک']]
BTS_Nodes.rename(columns={'کد سایت': 'Location'}, inplace = True)

Ticket_DetailOutage = Ticket_DetailOutage.merge(BTS_Nodes, left_on = ['Location'], right_on = ['Location'], how = 'left')

### Output

In [ ]:
Ticket_DetailOutage.to_csv('Outputs/Outage_Duration_Analysis(Seconds).csv', index=False, encoding='utf-8-sig') #Outage_Duration_Analysis(Seconds)